## PDE 1 - 2D

The problem setup is the same as in *2D_with_multiple_optimizers*.

We can calculate that without noise ($s=0$), the minimization problem becomes the following:
\begin{equation}
min_{\theta} \{\det((y^T \tilde{K}^{-1} y) \tilde{K})\},
\end{equation}
where $K = \sigma \tilde{K}$. For stability purposes we re-add the noise parameter to $\tilde{K}$. 

The gradient is given by
\begin{equation}
tr(A^{-1}\partial_{\theta_i}A),
\end{equation}
where $A = (y^T \tilde{K}^{-1} y) \tilde{K}$. 

This is equal to:
\begin{equation}
det((y^T \tilde{K}^{-1} y) \tilde{K}) \left[ tr(\tilde{K}^{-1} \partial_{\theta_i}\tilde{K}) - 2n \frac{y^T \tilde{K}^{-1} (\partial_{\theta_i}\tilde{K}) \tilde{K}^{-1} y}{y^T \tilde{K}^{-1} y} \right].
\end{equation}

#### Step 1: Simulate data

In [293]:
import time
import numpy as np
import sympy as sp
from scipy.linalg import solve_triangular
from scipy.optimize import minimize

In [294]:
# Global variables: x, y, n, y_u, y_f, s

*Parameters, that can be modified:*

In [295]:
# Number of data samples:
n = 4

# Noise of our data:
s = 1e-7

In [296]:
def simulate_data():
    x = np.random.rand(n)
    y = np.random.rand(n)
    y_u = np.multiply(x,x) + y
    y_f = 2*(np.multiply(x,x) + x + y)
    return (x,y,y_u,y_f)
(x,y,y_u,y_f) = simulate_data()
y_con = np.concatenate((y_u, y_f))

#### Step 2: Evaluate kernels

$k_{uu}(X_i, X_j; \theta) = exp(-\frac{1}{2l_x}(x_i-x_j)^2 - \frac{1}{2l_y}(y_i-y_j)^2)$

In [297]:
x_i, x_j, y_i, y_j, l_x, l_y, phi = sp.symbols('x_i x_j y_i y_j l_x l_y phi')
kuu_sym = sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_y)*((y_i - y_j)**2))
kuu_fn = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y), kuu_sym, "numpy")
def kuu(x, y, l_x, l_y):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], y[i], y[j], l_x, l_y)
    return k

$k_{ff}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} \mathcal{L}_{X_j}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \phi^2k_{uu} + \phi \frac{\partial}{\partial x_i}k_{uu} + \phi \frac{\partial^2}{\partial y_i^2}k_{uu} + \phi \frac{\partial}{\partial x_j}k_{uu} + \frac{\partial^2}{\partial x_i, x_j}k_{uu} + \frac{\partial^3}{\partial y_i^2 \partial x_j}k_{uu} + \phi \frac{\partial^2}{\partial y_j^2}k_{uu} + \frac{\partial^3}{\partial x_i \partial y_j^2}k_{uu} + \frac{\partial^4}{\partial y_i^2 \partial y_j^2}k_{uu}$

In [298]:
kff_sym = phi**2*kuu_sym \
        + phi*sp.diff(kuu_sym, x_i) \
        + phi*sp.diff(kuu_sym, y_i, y_i) \
        + phi*sp.diff(kuu_sym, x_j) \
        + sp.diff(kuu_sym, x_i, x_j) \
        + sp.diff(kuu_sym, y_i, y_i, x_j) \
        + phi*sp.diff(kuu_sym, y_j, y_j) \
        + sp.diff(kuu_sym, x_i, y_j, y_j) \
        + sp.diff(kuu_sym, y_i, y_i, y_j, y_j)
kff_fn = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kff_sym, "numpy")
def kff(x, y, l_x, l_y, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kff_fn(x[i], x[j], y[i], y[j], l_x, l_y, phi)
    return k

$k_{fu}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= \phi k_{uu} + \frac{\partial}{\partial x_i}k_{uu} + \frac{\partial^2}{\partial y_i^2}k_{uu}$

In [299]:
kfu_sym = phi*kuu_sym \
        + sp.diff(kuu_sym, x_i) \
        + sp.diff(kuu_sym, y_i, y_i)
kfu_fn = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kfu_sym, "numpy")
def kfu(x, y, l_x, l_y, phi):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kfu_fn(x[i], x[j], y[i], y[j], l_x, l_y, phi)
    return k

In [300]:
def kuf(x, y, l_x, l_y, phi):
    return kfu(x, y, l_x, l_y, phi).T

#### Step 3: Compute NLML (with Cholesky decomposition)

Implementing the covariance matrix K and its inverse

In [301]:
def K(l_x, l_y, phi):
    K_mat = np.block([
        [kuu(x, y, l_x, l_y) + s*np.eye(n), kuf(x, y, l_x, l_y, phi)],
        [kfu(x, y, l_x, l_y, phi), kff(x, y, l_x, l_y, phi) + s*np.eye(n)]
    ])
    return K_mat

In [302]:
def nlml(params):
    K_inv = np.zeros((2*n, 2*n))
    l_x_exp = np.exp(params[0])
    l_y_exp = np.exp(params[1]) 
    # phi = params[2]
    
    try:
        L = np.linalg.cholesky(K(l_x_exp, l_y_exp, params[2]))
        L_inv = solve_triangular(L, np.identity(2*n), lower=True) # Slight performance boost 
                                                                  # over np.linalg.inv
        K_inv = (L_inv.T).dot(L_inv)
        val = np.linalg.det((y_con @ K_inv @ y_con)*(L @ L.T))
        return val
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(K(l_x_exp, l_y_exp, params[2]))
        K_inv = (vt.T).dot(np.linalg.inv(np.diag(s_mat))).dot(u.T)
        val = np.prod((y_con @ K_inv @ y_con)*s_mat)
        return val

Can alternatively use the block inversion technique (see *2D_with_multiple_optimizers*):

In [303]:
def nlml_block(params):
    l_x_exp = np.exp(params[0])
    l_y_exp = np.exp(params[1]) 
    # phi = params[2]
    
    A = kuu(x, y, l_x_exp, l_y_exp) + s*np.eye(n)
    B = kfu(x, y, l_x_exp, l_y_exp, params[2]).T
    C = kff(x, y, l_x_exp, l_y_exp, params[2]) + s*np.eye(n)
    
    # Inversion of A
    A_inv = np.zeros((n, n))
    
    try:
        L = np.linalg.cholesky(A)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost 
                                                                # over np.linalg.inv
        A_inv = L_inv.T @ L_inv        
        det_A = (np.diag(L)**2).prod()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(A)
        A_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        det_A = s_mat.prod()        
        
    # Inversion of $C-B^T A^{-1} B$
    KA_inv = np.zeros((n, n))
    KA = C - B.T @ A_inv @ B
    
    try:
        L = np.linalg.cholesky(KA)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost 
                                                                # over np.linalg.inv
        KA_inv = L_inv.T @ L_inv
        det_KA = (np.diag(L)**2).prod()
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(KA)
        KA_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        det_KA = s_mat.prod()
        
    # Piecing it together
    T = A_inv @ B @ KA_inv
    factor = y_u @ (A_inv + T @ B.T @ A_inv) @ y_u - 2*y_u @ T @ y_f + y_f @ KA_inv @ y_f
    
    return np.linalg.det(factor*A)*np.linalg.det(factor*KA)

In [304]:
t0 = time.time()
m_block = minimize(nlml_block, np.random.rand(3), method="Nelder-Mead", 
             options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001})
t_Nelder_block = time.time() - t0

In [305]:
t0 = time.time()
m = minimize(nlml, np.random.rand(3), method="Nelder-Mead",
             options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001})
t_Nelder = time.time() - t0

***Computing the gradient***

In [306]:
kuu_sym_l_x, kuu_sym_l_y, kuu_sym_phi = [sp.diff(kuu_sym, l) for l in [l_x, l_y, phi]]
kfu_sym_l_x, kfu_sym_l_y, kfu_sym_phi = [sp.diff(kfu_sym, l) for l in [l_x, l_y, phi]]
kff_sym_l_x, kff_sym_l_y, kff_sym_phi = [sp.diff(kff_sym, l) for l in [l_x, l_y, phi]]

kuu_l_x = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y), kuu_sym_l_x, "numpy")
kff_l_x = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kff_sym_l_x, "numpy")
kfu_l_x = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kfu_sym_l_x, "numpy")

kuu_l_y = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y), kuu_sym_l_y, "numpy")
kff_l_y = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kff_sym_l_y, "numpy")
kfu_l_y = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kfu_sym_l_y, "numpy")

kuu_phi = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y), kuu_sym_phi, "numpy")
kff_phi = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kff_sym_phi, "numpy")
kfu_phi = sp.lambdify((x_i, x_j, y_i, y_j, l_x, l_y, phi), kfu_sym_phi, "numpy")

# Derivative of K w.r.t. l_x:
def K_l_x(x, y, l_x, l_y, phi):
    k = np.zeros([2*n, 2*n])
    for i in range(2*n):
        for j in range(2*n):
            if i<n and j<n:
                k[i,j] = kuu_l_x(x[i], x[j], y[i], y[j], l_x, l_y)
            if i<n and j>=n:
                k[i,j] = kfu_l_x(x[j-n], x[i], y[j-n], y[i], l_x, l_y, phi) # Transpose of kfu
            if i>=n and j<n:
                k[i,j] = kfu_l_x(x[i-n], x[j], y[i-n], y[j], l_x, l_y, phi)
            if i>=n and j>=n:
                k[i,j] = kff_l_x(x[i-n], x[j-n], y[i-n], y[j-n], l_x, l_y, phi)
    return k

# Derivative of K w.r.t. l_y:
def K_l_y(x, y, l_x, l_y, phi):
    k = np.zeros([2*n, 2*n])
    for i in range(2*n):
        for j in range(2*n):
            if i<n and j<n:
                k[i,j] = kuu_l_y(x[i], x[j], y[i], y[j], l_x, l_y)
            if i<n and j>=n:
                k[i,j] = kfu_l_y(x[j-n], x[i], y[j-n], y[i], l_x, l_y, phi) # Transpose of kfu
            if i>=n and j<n:
                k[i,j] = kfu_l_y(x[i-n], x[j], y[i-n], y[j], l_x, l_y, phi)
            if i>=n and j>=n:
                k[i,j] = kff_l_y(x[i-n], x[j-n], y[i-n], y[j-n], l_x, l_y, phi)
    return k

# Derivative of K w.r.t. phi:
def K_phi(x, y, l_x, l_y, phi):
    k = np.zeros([2*n, 2*n])
    for i in range(2*n):
        for j in range(2*n):
            if i<n and j<n:
                k[i,j] = kuu_phi(x[i], x[j], y[i], y[j], l_x, l_y)
            if i<n and j>=n:
                k[i,j] = kfu_phi(x[j-n], x[i], y[j-n], y[i], l_x, l_y, phi) # Transpose of kfu
            if i>=n and j<n:
                k[i,j] = kfu_phi(x[i-n], x[j], y[i-n], y[j], l_x, l_y, phi)
            if i>=n and j>=n:
                k[i,j] = kff_phi(x[i-n], x[j-n], y[i-n], y[j-n], l_x, l_y, phi)
    return k

In [307]:
def K_inv_and_det(l_x, l_y, phi):
    
    A = kuu(x, y, l_x, l_y) + s*np.eye(n)
    B = kfu(x, y, l_x, l_y, phi).T
    C = kff(x, y, l_x, l_y, phi) + s*np.eye(n)
    
    # Inversion of A
    A_inv = np.zeros((n, n))
    
    try:
        L = np.linalg.cholesky(A)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost 
                                                                # over np.linalg.inv
        A_inv = L_inv.T @ L_inv
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(A)
        A_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        
    # Inversion of $C-B^T A^{-1} B$
    KA_inv = np.zeros((n, n))
    KA = C - B.T @ A_inv @ B
    
    try:
        L = np.linalg.cholesky(KA)
        L_inv = solve_triangular(L, np.identity(n), lower=True) # Slight performance boost 
                                                                # over np.linalg.inv
        KA_inv = L_inv.T @ L_inv
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(KA)
        KA_inv = vt.T @ np.linalg.inv(np.diag(s_mat)) @ u.T
        
    # Piecing it together
    T = A_inv @ B @ KA_inv
        
    K_inv = np.block([
        [A_inv + T @ B.T @ A_inv, -T],
        [-T.T, KA_inv]
    ])
    
    f = y_con @ K_inv @ y_con
    
    det_fK = np.linalg.det(f*A)*np.linalg.det(f*KA)
    
    return K_inv, det_fK

In [308]:
# Setting the gradient:
def grad_L(l_x, l_y, phi):
    
    K_inv, det_fK = K_inv_and_det(l_x, l_y, phi)
    
    K_1 = K_l_x(x, y, l_x, l_y, phi)
    K_2 = K_l_y(x, y, l_x, l_y, phi)
    K_3 = K_phi(x, y, l_x, l_y, phi)
    
    w = K_inv @ y_con

    comp_1 = det_fK*(np.trace(K_inv @ K_1) - (2*n*w.T @ K_1 @ w) / (y_con @ w))
    comp_2 = det_fK*(np.trace(K_inv @ K_2) - (2*n*w.T @ K_2 @ w) / (y_con @ w))
    comp_3 = det_fK*(np.trace(K_inv @ K_3) - (2*n*w.T @ K_3 @ w) / (y_con @ w))
    
    return [comp_1, comp_2, comp_3]

def grad_nlml(par):
    # l_x = par[0]
    # l_y = par[1]
    # phi = par[2]
    M = np.diag([np.exp(par[0]), np.exp(par[1]), 1])
    grad = np.array([grad_L(np.exp(par[0]), np.exp(par[1]), par[2])]).dot(M)
    
    # Scipy.minimize wants an array as the gradient, not a matrix. 
    # We therefore remove any one-dimensional entries of the grad by squeezing.
    return np.squeeze(np.asarray(grad))

Running conjugate gradient:

In [309]:
Nfeval = 1
def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f}   {2: 3.6f}   {3: 3.6f}'.format(Nfeval, Xi[0], Xi[1], Xi[2]))
    Nfeval += 1

In [310]:
import conjugate_gradient._minimize as mi

In [311]:
t1 = time.time()
m_CG = mi.minimize(nlml, np.random.rand(3), jac=grad_nlml, method='CG')
t_CG = time.time() - t1

In [312]:
print('The time it took Nelder-Mead with block-inversion to converge:', t_Nelder_block, 'seconds')
print('The time it took Nelder-Mead to converge:                     ', t_Nelder, 'seconds')
print('The time it took non-linear Conjugate Gradient to converge:   ', t_CG, 'seconds')

The time it took Nelder-Mead with block-inversion to converge: 0.18849539756774902 seconds
The time it took Nelder-Mead to converge:                      0.23704886436462402 seconds
The time it took non-linear Conjugate Gradient to converge:    0.5568571090698242 seconds


In [313]:
print('The inferred parameter with Nelder-Mead with block-inversion is:', m_block.x[2])
print('The inferred parameter with Nelder-Mead is:                     ', m.x[2])
print('The inferred parameter with non-linear Conjugate Gradient is:   ', m_CG.x[2])

The inferred parameter with Nelder-Mead with block-inversion is: 2.0116897972421492
The inferred parameter with Nelder-Mead is:                      2.0116696984130775
The inferred parameter with non-linear Conjugate Gradient is:    2.0195302066585694
